# Импорт всех библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import keras
import timeit
import time
import cv2
import os
import pathlib
import random
import pandas as pd
from PIL import Image
from numpy import asarray

from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate
from tensorflow.keras.metrics import BinaryIoU
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Выделение файлов из хранилища и преобразование

In [ ]:
path_img, path_clust = '/content/drive/MyDrive/training_data/data/Neural_network_img_cl/nu_int_dn2_img', '/content/drive/MyDrive/training_data/data/Neural_network_img_cl/nu_int_dn2_clust'

res_img, res_clust = os.listdir(path_img), os.listdir(path_clust)

img_list, clust_list = [], []

for item in res_img:
    path_img_dir = os.path.join(path_img, item)
    for file in os.listdir(path_img_dir):
        img_list.append(os.path.join(path_img_dir, file))

for item in res_clust:
    path_clust_dir = os.path.join(path_clust, item)
    for file in os.listdir(path_clust_dir):
        clust_list.append(os.path.join(path_clust_dir, file))

img_list.sort()
clust_list.sort()

In [ ]:
images_l = []
masks_l = []

count = 0

for path in img_list:
  count += 1
  if count % 30 == 0:
    print(count)
  img = Image.open(path)
  numpydata_img = asarray(img)
  images_l.append(numpydata_img)

count = 0
for path in clust_list:
  count += 1
  if count % 30 == 0:
    print(count)
  img = Image.open(path)
  numpydata_clust = asarray(img)
  masks_l.append(numpydata_clust)


images = np.array(images_l)
masks = np.array(masks_l)

images = np.expand_dims(images, axis=-1)
masks = np.expand_dims(masks, axis=-1)

images = images / 255
masks = np.array(masks > 0.28, dtype='float')

In [ ]:
print(images.shape)

print(masks.shape)

In [ ]:
images[0].max(), images.min()

# ImageDataGenerator

In [ ]:
gen = ImageDataGenerator().flow(x=images, y=masks, batch_size=4)

In [ ]:
x_batch, y_batch = next(gen)

In [ ]:
x_batch.shape, y_batch.shape

In [ ]:
for i in range(0, 4):
    plt.figure(figsize=(20,12))
    plt.subplot(121)
    plt.imshow(x_batch[i,:,:,0], cmap="gray")
    plt.subplot(122)
    plt.imshow(y_batch[i,:,:,0], cmap="gray")
    plt.show()

In [ ]:
x_batch.max(), x_batch.min(), y_batch.max(), y_batch.min(), np.unique(y_batch)

# Создание нейросети и её обучение (с сохранением лучшего результата)

In [ ]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

#Блок кодировки (сжатия)


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
model = build_unet((1024, 1280, 1))
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='BinaryFocalCrossentropy', metrics=[ BinaryIoU(target_class_ids=[0])])

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model.h5",
    save_weights_only=True,
    monitor='binary_io_u',
    mode='max',
    save_best_only=True)


In [ ]:
model.fit(gen, epochs=3, callbacks=[model_checkpoint_callback])

# Результаты обучения

In [ ]:
y_pred = model.predict(x_batch)
y_pred_mod = y_pred > 0.5

In [ ]:
y_pred.max(), y_pred.min()

In [ ]:
for i in range(len(x_batch)):
    m = BinaryIoU(target_class_ids=[0])
    m.update_state(y_pred_mod[i, :, :, 0], y_batch[i, :, :, 0])
    print(m.result().numpy())

    plt.figure(figsize=(20,12))
    plt.subplot(131)
    plt.imshow(x_batch[i,:,:,0], cmap="gray")
    plt.subplot(132)
    plt.imshow(y_batch[i,:,:,0], cmap="gray")
    plt.subplot(133)
    plt.imshow(y_pred_mod[i,:,:,0], cmap="gray")
    plt.show()
    print("\n")


In [ ]:
y_pred_mod.max(), y_pred_mod.min()

# Сохранение модели

In [ ]:
model_file = model.save("model.h5")

# Загрузка обученной модели

In [ ]:
loaded_model = keras.models.load_model("/content/drive/MyDrive/Project_Altair_10thGrade/model.h5")

loaded_model.summary()

# Проверка сети на другом наборе

In [ ]:
path_img, path_clust = '/content/drive/MyDrive/training_data/check_data/img', '/content/drive/MyDrive/training_data/check_data/clust'

res_img, res_clust = os.listdir(path_img), os.listdir(path_clust)

img_check_list, clust_check_list = [], []

for item in res_img:
    path_img_dir = os.path.join(path_img, item)
    for file in os.listdir(path_img_dir):
        img_check_list.append(os.path.join(path_img_dir, file))

for item in res_clust:
    path_clust_dir = os.path.join(path_clust, item)
    for file in os.listdir(path_clust_dir):
        clust_check_list.append(os.path.join(path_clust_dir, file))

img_check_list.sort()
clust_check_list.sort()

In [ ]:
img_check_list

In [ ]:
clust_check_list

In [ ]:
images_check_l = []
masks_check_l = []

count = 0

for path in img_check_list:
  count += 1
  if count % 10 == 0:
    print(count)
  img = Image.open(path)
  numpydata_img = asarray(img)
  images_check_l.append(numpydata_img)

count = 0
for path in clust_check_list:
  count += 1
  if count % 10 == 0:
    print(count)
  img = Image.open(path)
  numpydata_clust = asarray(img)
  masks_check_l.append(numpydata_clust)


images_exam = np.array(images_check_l)
masks_exam = np.array(masks_check_l)

images_exam = np.expand_dims(images_exam, axis=-1)
masks_exam = np.expand_dims(masks_exam, axis=-1)

images_exam = images_exam / 255
masks_exam = np.array(masks_exam > 0.28, dtype='float')

In [ ]:
images_exam.shape, masks_exam.shape

In [ ]:
images_exam[0].max(), images_exam.min()

In [ ]:
gen_exam = ImageDataGenerator().flow(x=images_exam, y=masks_exam, batch_size=4)

In [ ]:
x_batch_exam, y_batch_exam = next(gen_exam)

In [ ]:
x_batch_exam.shape, y_batch_exam.shape

In [ ]:
for i in range(0, 4):
    plt.figure(figsize=(20,12))
    plt.subplot(121)
    plt.imshow(x_batch_exam[i,:,:,0], cmap="gray")
    plt.subplot(122)
    plt.imshow(y_batch_exam[i,:,:,0], cmap="gray")
    plt.show()

In [ ]:
start_time = time.time()

one_pict_pred = loaded_model.predict(x_batch_exam)

print('One image is generated in', round((time.time() - start_time) / len(img_check_list) * 1000), "ms")

In [ ]:
y_pred_exam = loaded_model.predict(x_batch_exam)
y_pred_exam_mod = y_pred_exam > 0.5

In [ ]:
x_batch_exam.max(), x_batch_exam.min(), y_batch_exam.max(), y_batch_exam.min(), np.unique(y_batch_exam)

In [ ]:
y_pred_exam.max(), y_pred_exam.min(), np.unique(y_pred_exam)

In [ ]:
for i in range(len(x_batch_exam)):
    m = BinaryIoU(target_class_ids=[0])
    m.update_state(y_pred_exam_mod[i, :, :, 0], y_batch_exam[i, :, :, 0])
    print(m.result().numpy())
    

    plt.figure(figsize=(20, 12))
    plt.subplot(131)
    plt.imshow(x_batch_exam[i,:,:,0], cmap="gray")
    plt.subplot(132)
    plt.imshow(y_batch_exam[i,:,:,0], cmap="gray")
    plt.subplot(133)
    plt.imshow(y_pred_exam_mod[i,:,:,0], cmap="gray")
    plt.show()
    print("\n")

# Intersection over Union

In [ ]:
num_pict = random.randint(0, 4)
iou_real = y_batch_exam[num_pict,:,:,0]
iou_pred = y_pred_exam_mod[num_pict,:,:,0]

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(121)
plt.imshow(iou_real, cmap="gray")
plt.subplot(122)
plt.imshow(iou_pred, cmap="gray")
plt.show()

In [ ]:
iou_real.shape, iou_pred.shape

In [ ]:
iou_real = np.expand_dims(iou_real, -1)

iou_pred = np.expand_dims(iou_pred, -1)

In [ ]:
plt.imshow(np.array(iou_real < 0.5, dtype='float')[:,:,0], cmap='gray')

In [ ]:
plt.imshow(np.array(iou_pred < 0.7, dtype='float')[:,:,0], cmap='gray')

In [ ]:
true_mask = np.array(iou_real < 0.5, dtype='float')
predicted_mask = np.array(iou_pred < 0.7, dtype='float')

In [ ]:
plt.imshow(true_mask[:,:,0] + predicted_mask[:,:,0], cmap='gray')

In [ ]:
true_mask_rgb = np.array(np.concatenate([true_mask * 255, true_mask, true_mask], -1), dtype='int')
predicted_mask_rgb = np.array(np.concatenate([predicted_mask, predicted_mask * 255, predicted_mask], -1), dtype='int')


In [ ]:
(true_mask_rgb + predicted_mask_rgb).shape

In [ ]:
plt.figure(figsize=(300, 180))
plt.imshow((true_mask_rgb + predicted_mask_rgb))
